In [1]:
# Install these packages for using langchain and reading documents
#!pip install --q unstructured langchain

In [1]:
from langchain_community.document_loaders import UnstructuredPDFLoader

### Reading Pdf Data

In [2]:
# pdf file location
localpath = "C:/Drive D/Documents/Why should I trust you.pdf"

# read the file from the location
try:
    pdf_loader = UnstructuredPDFLoader(file_path = localpath)
    pdf_data = pdf_loader.load()
except Exception as e:
    print(e)

### Vector Embeddings

In [7]:
# to download the nomic embed text model into local
# !ollama pull nomic-embed-text

In [8]:
# to download the gemma:2b model into local
# !ollama pull gemma:2b

In [6]:
!ollama list

NAME                   	ID          	SIZE  	MODIFIED       
gemma:2b               	b50d6c999e59	1.7 GB	36 seconds ago	
nomic-embed-text:latest	0a109f422b47	274 MB	3 minutes ago 	


In [12]:
# to install chromadb in local
#!pip install --q chromadb

In [13]:
# to install langchain text splitters in local
#!pip install -q langchain-text-splitters

In [14]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [20]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2000, chunk_overlap = 20)
chunks = text_splitter.split_documents(pdf_data)

In [23]:
# Add the data into vector DB
vector_db = Chroma.from_documents(
    documents = chunks,
    embedding = OllamaEmbeddings(model = "nomic-embed-text", show_progress = True),
    collection_name = "local-chroma-rag"
)

OllamaEmbeddings: 100%|██████████| 38/38 [02:36<00:00,  4.13s/it]


#### Retrieval

In [28]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [29]:
# llm from google
local_model = "gemma:2b"
llm = ChatOllama(model = local_model)

In [30]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [31]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [32]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [33]:
chain.invoke(input(""))

 What this document is about?


OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


'The document is about explaining how to select and trust models for machine learning tasks. It gives insights into selecting the most relevant features for explanations, and helps users assess the trust of predictions.'